In [1]:
import requests
from bs4 import BeautifulSoup

HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
}

In [10]:
class BBCGoodfoodScraper():
    recipes = {}
    def __init__(self):
        self.links = []
        self.rootUrl = 'https://www.bbcgoodfood.com'
        self.recipes['recipes'] = {}

    def getRecipesLinksFromPage(self, url):
        page = requests.get(self.rootUrl+url, headers = HEADERS)
        soup = BeautifulSoup(page.content, 'html.parser')

        for recipe in soup.find_all('h3', class_='teaser-item__title'):
            for link in recipe.find_all('a'):
                self.links.append(link.get('href'));
        nextUrl = soup.find('a', attrs={'title':'Go to next page'})
        try:
            return(nextUrl.get('href'))
        except Exception as inst:
            return None

    def getAllRecipeLinks(self, nextUrl):
        while(nextUrl is not None):
            nextUrl = self.getRecipesLinksFromPage(nextUrl);


    def getRecipe(self, url, idx):
        recipe = {}
        recipe['details'] = {}
        recipe['ingredients'] = {}
        recipe['method'] = {}
        page = requests.get(self.rootUrl+url, headers = HEADERS)
        soup = BeautifulSoup(page.content, 'html.parser')
        recipe_title = soup.find('h1', attrs={'class':'recipe-header__title', 'itemprop':'name'}).text
        recipe_details = soup.find('div', attrs={'class':'recipe-details'})
        recipe_ingredients = soup.find('div', attrs={'class':'ingredients-list__content'})
        recipe_method = soup.find('div', attrs={'class':'method'})
        recipe['title'] = recipe_title
        recipe['details']['prep_time'] = recipe_details.find_all('span', attrs={'class':'mins'})[0].text
        recipe['details']['cook_time'] = recipe_details.find_all('span', attrs={'class':'mins'})[1].text
        recipe['details']['difficulty'] = recipe_details.find('span', attrs={'class':'recipe-details__text'}).text
        count = 0
        for ingredient_text in recipe_ingredients.find_all('li', attrs={'class':'ingredients-list__item'}):
            recipe['ingredients'][count] = {}
            recipe['ingredients'][count]['name'] = list(ingredient_text.stripped_strings)[0] if len( list(ingredient_text.stripped_strings)) < 2 else list(ingredient_text.stripped_strings)[0] + ' ' + list(ingredient_text.stripped_strings)[1] 
            count += 1
        recipe['ingredients']['count'] = count
        count = 0
        for method_step in recipe_method.stripped_strings:
            recipe['method'][count] = method_step
            count += 1
        recipe['method']['count'] = count
        self.recipes['recipes'][idx] = recipe

    def getAllRecipes(self):
        count = 0
        for link in self.links:
            try:
                self.getRecipe(link, count)
                count += 1
            except Exception as inst:
                print(count)
                print(link)
        self.recipes['count'] = count

In [11]:
scraper = BBCGoodfoodScraper()
scraper.getAllRecipeLinks('/recipes/collection/under-20-minutes')

In [12]:
scraper.getAllRecipes()

2
/recipes/amaretti-affogato
3
/recipes/smoked-mackerel-courgette-butter-bean-salad
10
/recipes/chocolate-fridge-cake
16
/recipes/cheats-laksa
16
/recipes/beetroot-feta-grain-salad
28
/recipes/1652/curry-in-a-hurry
30
/recipes/1940/20minute-seafood-pasta-
30
/recipes/3078/20minute-pork-panfry


In [9]:
scraper.recipes

{'count': 1,
 'recipes': {0: {'details': {'cook_time': '10 mins',
    'difficulty': ' Easy ',
    'prep_time': '10 mins'},
   'ingredients': {0: {'name': '200g cooked beetroot'},
    1: {'name': '70g feta'},
    2: {'name': '2 limes'},
    3: {'name': '2 skin-on salmon'},
    'count': 4},
   'method': {0: 'Chop the beetroot and feta into small cubes and mix with the juice and zest of one lime and some seasoning.',
    1: 'Season the salmon. Heat 2 tbsp of oil in a',
    2: 'nonstick frying pan',
    3: 'over a high heat. When hot add the salmon, skin-side down, and cook for 3 mins. Flip over, turn the heat down and cook for a further 4-5 mins. Serve with the beetroot salsa and the remaining lime, cut into wedges.',
    'count': 4},
   'title': 'Salmon with beetroot, feta & lime salsa'}}}

In [ ]:
#if __name__ == "__main__":
#    scraper = BBCGoodfoodScraper()
#    scraper.getAllRecipeLinks('/recipes/collection/under-20-minutes')
#    scraper.getAllRecipes()
